## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [1]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict
import sqlite3

# Feel free to include your text patterns functions
#from GroupComparison import tokenize, get_patterns

from sklearn.model_selection import train_test_split

In [5]:
# accessing previous assignment did not work on my pc tried many troubleshooting things instead went on this route:

#%run -i GroupComparison.ipynb

In [6]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [7]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

query_results = convention_cur.execute(
                            '''
                            SELECT party, text FROM conventions
                            ''')

for row in query_results :
    party, text = row
    # Clean and tokenize the text
    
    tokenized_text = prepare(text,pipeline=my_pipeline)
    tokenized_text = [token.lower() for token in tokenized_text]
    # Append the tokenized text and party to the convention_data list
    convention_data.append([tokenized_text, party])





In [8]:
df = pd.DataFrame(convention_data, columns=["text", "party"])

Let's look at some random entries and see if they look right. 

In [9]:
random.choices(convention_data,k=10)

[[['grew',
   'willow',
   'grove',
   'pennsylvania',
   'right',
   'outside',
   'philly',
   'mom',
   'englishscottish',
   'dad',
   'italian',
   'grandmother',
   'made',
   'homemade',
   'noodles',
   'would',
   'hang',
   'shed',
   'drying',
   'noodles'],
  'Democratic'],
 [['established',
   'taskforce',
   'called',
   'operation',
   'lady',
   'justice',
   'address',
   'problem',
   'president',
   'also',
   'provided',
   '273',
   'million',
   'improve',
   'public',
   'safety',
   'support',
   'victims',
   'crime',
   'native',
   'american',
   'community',
   'result',
   'coldcase',
   'office',
   'recently',
   'opened',
   'gila',
   'river',
   'indian',
   'community',
   'president',
   'reactivated',
   'white',
   'house',
   'council',
   'native',
   'american',
   'affairs',
   'promote',
   'economic',
   'development',
   'rural',
   'prosperity',
   'indian',
   'country',
   'president',
   'trump',
   'also',
   'strengthened',
   'supreme

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [10]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2385 as features in the model.


In [11]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Your code here
    ret_dict = dict()
    for word in text:
    	if word in fw:
        	ret_dict[word] = True
        

    
    return(ret_dict)

In [46]:
#assert(len(feature_words)>0)
#assert(conv_features("donald is the president",feature_words)==
       #{'donald':True,'president':True})
#assert(conv_features("people are american in america",feature_words)==
                     #{'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [13]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [14]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [15]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.496


In [16]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                  defund = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

`--> It may be hard to have prose with the data printed above---based on only printing 25 words at a time---, but we can say that the Republican party includes more words I would consider soundsbites or keywords in the context of an election (ex. china, isis, drug,enemy, etc).As an example China was mentioned 25.8 and Isis 13.0 more times than by Democrats. While the data above seems like the Democratic party strayed away from that tactic and tried to focus on another agenda (voting, and climate) were they mentioned it 23.8 and 17.8 more times than Republicans. Either way, I understand this is not about polictics and more on exploring the NB model. On a intersting note I see another way people can make charts and stats claiming bias whithout showing the whole picture. `





## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [17]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [31]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [32]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.


for row in results :
    name, party, text = row
    # Clean and tokenize the text
    text = text.decode()
    tokenized_text2 = prepare(text,pipeline=my_pipeline)
    tokenized_text2 = [token.lower() for token in tokenized_text2]
    # Append the tokenized text and party to the convention_data list
    tweet_data.append([tokenized_text2, party])


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [33]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)


In [34]:
tweet_data_sample

[[['earlier',
   'today',
   'spoke',
   'house',
   'floor',
   'abt',
   'protecting',
   'health',
   'care',
   'women',
   'praised',
   'ppmarmonte',
   'work',
   'central',
   'coast',
   'httpstcowqgtrzt7vv'],
  'Democratic'],
 [['go', 'tribe', '#', 'rallytogether', 'httpstco0nxutfl9l5'], 'Democratic'],
 [['apparently',
   'trump',
   'thinks',
   'easy',
   'students',
   'overwhelmed',
   'crushing',
   'burden',
   'debt',
   'pay',
   'student',
   'loans',
   '#',
   'trumpbudget',
   'httpstcockyqo5t0qh'],
  'Democratic'],
 [['grateful',
   'first',
   'responders',
   'rescue',
   'personnel',
   'firefighters',
   'police',
   'volunteers',
   'working',
   'tirelessly',
   'keep',
   'people',
   'safe',
   'provide',
   'muchneeded',
   'help',
   'putting',
   'lives',
   'line',
   'httpstcoezpv0vmiz3'],
  'Republican'],
 [['lets', 'make', 'even', 'greater', '#', 'kag', '🇺🇸', 'httpstcoy9qozd5l2z'],
  'Republican'],
 [['1hr',
   'cavs',
   'tie',
   'series',
   '22

In [43]:
word_cutoff = 5

tokens = [w for t, p in tweet_data for w in t]


word_dist2 = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2385 as features in the model.


In [44]:

for tweet, party in tweet_data_sample :
    estimated_party = classifier.classify(conv_features(tweet, feature_words))
    # Fill in the right-hand side above with code that estimates the actual party
    
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: ['earlier', 'today', 'spoke', 'house', 'floor', 'abt', 'protecting', 'health', 'care', 'women', 'praised', 'ppmarmonte', 'work', 'central', 'coast', 'httpstcowqgtrzt7vv']
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: ['go', 'tribe', '#', 'rallytogether', 'httpstco0nxutfl9l5']
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: ['apparently', 'trump', 'thinks', 'easy', 'students', 'overwhelmed', 'crushing', 'burden', 'debt', 'pay', 'student', 'loans', '#', 'trumpbudget', 'httpstcockyqo5t0qh']
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: ['grateful', 'first', 'responders', 'rescue', 'personnel', 'firefighters', 'police', 'volunteers', 'working', 'tirelessly', 'keep', 'people', 'safe', 'provide', 'muchneeded', 'help', 'putting', 'lives', 'line', 'httpstcoezpv0vmiz3']
Actual party is Republican and our classifer says Republican.

Here

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [37]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them.
    fw = conv_features(tweet, feature_words)
   
    # get the estimated party
    estimated_party = classifier.classify(fw)
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [38]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3742, 'Democratic': 536}),
             'Democratic': defaultdict(int,
                         {'Republican': 4846, 'Democratic': 878})})

In [39]:
predictions = [(conv_features(text,feature_words), party) for (text, party) in tweet_data]
print(nltk.classify.accuracy(classifier, predictions))

0.46139807659902266


### Reflections

While our classifier is able to classify tweets belonging to a Democrat or Republican candidate, our accuray leaves a lot to desire. With an accuracy of 0.46 the model does not inspire confidence as its below a coin toss---50% chance--- of getting it right. Although it was a great exercise that joint the previous assignment and is a taste of real world pulling data and classifying it. If given more time this project can benefit from a better removable of text as some emojis and urls were tokenized. Additionally, the classifier suffered from an unbalanced dataset. We could downsample or upsample the data to address the class imbalance. As well as considering evaluating the project in other metrics beyond accuracy--- precison and recall--- to better understand the model's performance. 
